In [1]:
#使用ResNet18初始化，使用自定义模型训练（多次多头注意力迭代）
#更改Wx输出后的大小
#

In [2]:
import matplotlib.pyplot as plt
import math
from torch.cuda.amp import autocast 
from torch.cuda.amp import GradScaler
import torch
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch import optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
import torch.nn as nn
import torchvision
from torchsummary import summary
import time
from torch.nn import init
from typing import Union, List, Dict, Any, Optional, cast
import warnings
warnings.filterwarnings("ignore")

In [3]:
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

In [4]:
#保存训练数据和模型
data_csv_path = "D:\\OneModel\\qyxx-eca_net-KA.csv"   #修改此处文件名 
model_save_path = "D:\\OneModel\\qyxx-eca_net-KA.pkl"  #修改此处文件名

In [5]:
train_path = "D:\\Dataset\\inputdataset\\train"
val_path = "D:\\Dataset\\inputdataset\\val"
#模型批次大小
batch_size = 128
resume = True
#动态学习率，学习率和循环次数增加
lr = 5e-4
epochs = 200
D_epoch = 0 
best_acc  = 0
print("epochs:",epochs,"learning_rate:",lr,"batch_size:",batch_size)

epochs: 200 learning_rate: 0.0005 batch_size: 128


In [6]:
#设备选取
flag = torch.cuda.is_available()
if flag:
    print("GPU")
else:
    print("CPU")
ngpu = 1
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
# device = torch.device("cpu")
#查看显卡名称
#torch.cuda.get_device_name()
print("divice is ", device)

#数据预处理（建议提前resize，减少每次资源的损失）放大到112x112 ，随机水平翻转
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

#增加不同种transform，预测集中去除随机翻转
val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

GPU
divice is  cuda:0


In [7]:
class eca_layer(nn.Module):
    """Constructs a ECA module.
    Args:
        channel: Number of channels of the input feature map
        k_size: Adaptive selection of kernel size
    """
    def __init__(self, channel, k_size=3):
        super(eca_layer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv = nn.Conv1d(1, 1, kernel_size=k_size, padding=(k_size - 1) // 2, bias=False) 
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # feature descriptor on the global spatial information
        y = self.avg_pool(x)

        # Two different branches of ECA module
        y = self.conv(y.squeeze(-1).transpose(-1, -2)).transpose(-1, -2).unsqueeze(-1)

        # Multi-scale information fusion
        y = self.sigmoid(y)

        return x * y.expand_as(x)
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class ECABasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, k_size=3):
        super(ECABasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes, 1)
        self.bn2 = nn.BatchNorm2d(planes)
        self.eca = eca_layer(planes, k_size)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.eca(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ECABottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, k_size=3):
        super(ECABottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.eca = eca_layer(planes * 4, k_size)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)
        out = self.eca(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, k_size=[3, 3, 3, 3]):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0], int(k_size[0]))
        self.layer2 = self._make_layer(block, 128, layers[1], int(k_size[1]), stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], int(k_size[2]), stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], int(k_size[3]), stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, k_size, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, k_size))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, k_size=k_size))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


def eca_resnet18(k_size=[3, 5, 7, 7], num_classes=1_000, pretrained=False):
    """Constructs a ResNet-18 model.
    Args:
        k_size: Adaptive selection of kernel size
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        num_classes:The classes of classification
    """
    model = ResNet(ECABasicBlock, [2, 2, 2, 2], num_classes=num_classes, k_size=k_size)
    model.avgpool = nn.AdaptiveAvgPool2d(1)
    return model


In [8]:
from torch.nn.parallel import DistributedDataParallel
model_path = "C:\\Users\\sk\\Desktop\\eca_resnet18_k3577.pth.tar"
eca_net = eca_resnet18()
checkpoint = torch.load(model_path)
eca_net.load_state_dict({k.replace('module.',''):v for k,v in checkpoint['state_dict'].items()})
del checkpoint
for p in eca_net.parameters():
    p.requires_grad=False
eca_net

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ECABasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (eca): eca_layer(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (conv): Conv1d(1, 1, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
        (sigmoid): Sigmoid()
      )
    )
    (1): ECABasicBlock(
      (conv1): 

In [9]:
class SKNet(nn.Module):
    def __init__(self, num_class=7):
        super(SKNet, self).__init__()
        self.features = nn.Sequential(*list(eca_net.children())[:-2])
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(512, num_class)

    def forward(self, x):
        
        x = self.features(x)
        out = self.avgpool(x)
        out = torch.flatten(out,1)
        out = self.fc(out)
        return out

In [10]:
#使用torchvision.datasets.ImageFolder读取数据集指定train和test文件夹
train_data = torchvision.datasets.ImageFolder(train_path, transform=train_transform)
#drop_last舍弃未满一个批次的数据        num_workers工作区一般设置为GPU个数的4倍
data0_train = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True,num_workers=4)
print(train_data)  #输出训练集相关
val_data = torchvision.datasets.ImageFolder(val_path, transform=val_transform)
data1_val = DataLoader(val_data, batch_size=batch_size, shuffle=True,drop_last=True,num_workers=4)
print(val_data)  #输出测试集相关

print(train_data.classes)  #根据分的文件夹的名字来确定的类别
print(train_data.class_to_idx) #按顺序为这些类别定义索引为0,1...
print()
print(val_data.classes)
print(val_data.class_to_idx)
print()


Dataset ImageFolder
    Number of datapoints: 783
    Root location: D:\Dataset\inputdataset\train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
Dataset ImageFolder
    Number of datapoints: 198
    Root location: D:\Dataset\inputdataset\val
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
['anger', 'contempt', 'disgust', 'fear', 'happy', 'sadness', 'surprise']
{'anger': 0, 'contempt': 1, 'disgust': 2, 'fear': 3, 'happy': 4, 'sadness': 5, 'surprise': 6}

['anger', 'contempt', 'disgust', 'fear', 'happy', 'sadness', 'surprise']
{'anger': 

In [11]:
#to(device)将模型加入GPU中加速计算
model = SKNet().to(device)
#设置优化器

params = list(model.parameters()) 
optimizer = optim.AdamW(params, lr=lr)

# optimizer = optim.AdamW(model.parameters(), lr=lr)
#设置损失函数
criteon = nn.CrossEntropyLoss().to(device)
#余弦衰减学习率
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=80, eta_min=0)
#形如TensorFlow中的summary函数输出模型参数
summary(model, input_size=[(3, 224, 224)], batch_size=batch_size, device="cuda")
print()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [128, 64, 112, 112]           9,408
       BatchNorm2d-2        [128, 64, 112, 112]             128
              ReLU-3        [128, 64, 112, 112]               0
         MaxPool2d-4          [128, 64, 56, 56]               0
            Conv2d-5          [128, 64, 56, 56]          36,864
       BatchNorm2d-6          [128, 64, 56, 56]             128
              ReLU-7          [128, 64, 56, 56]               0
            Conv2d-8          [128, 64, 56, 56]          36,864
       BatchNorm2d-9          [128, 64, 56, 56]             128
AdaptiveAvgPool2d-10            [128, 64, 1, 1]               0
           Conv1d-11               [128, 1, 64]               3
          Sigmoid-12            [128, 64, 1, 1]               0
        eca_layer-13          [128, 64, 56, 56]               0
             ReLU-14          [128, 64,

In [12]:
#测试函数
def evalute_(model,val_loader):
    model.eval()
    test_loss2 = 0.0
    test_corrects2 = 0.0
    number = 0
    for batchidx, (x, label) in enumerate(val_loader):
#         print(number)
    #torch.cuda.empty_cache()  #清除非必要GPU缓存，但是我建议不要在训练中使用此句，这可能会损失你相当多的时间
        number = number + 1
        x, label = x.to(device), label.to(device)
        #测试函数中加入no_grad()，如果不加会增加计算和显存
        with torch.no_grad():
            y1 = model(x)
            #虽然可以直接使用max函数，但是我建议在y1的比较重你最好使用F.softmax(y1,dim=1)，这样可能会有更好的效果，我在训练中使用了它
            _, preds1 = torch.max(F.softmax(y1,dim=1), 1)
            loss = criteon(y1, label)  
            
            test_loss2 += loss.item()*batch_size
            test_corrects2 += torch.sum(preds1 == label.data)
    #由于使用了最后一次抛弃，我不能使用全部测试集作为分母，这样会使最后的准确率变小
    test_loss1 = test_loss2 / (number*batch_size)
    test_acc1 = test_corrects2.double() / (number*batch_size)
#     print("TestDataset loss is ", test_loss1,"TestDataset accuracy is ",test_acc1)
    return test_acc1, test_loss1
print("执行结束")

执行结束


In [13]:
#关于AMP自动精度求解，我也并不是很熟悉，只能使用官方给的实例进行照葫芦画瓢。
scaler = torch.cuda.amp.GradScaler()
torch.cuda.empty_cache()
for epoch in range(D_epoch, epochs):
    time_one = time.time()                         #标记训练开始时间戳
    train_acc1 = 0.0
    train_loss1 = 0.0
    train_acc = 0.0
    train_loss = 0.0
    
    val_acc = 0.0
    val_loss = 0.0
    number = 0
    model.train()
    print("epoch:",epoch)

    for batchidx , (x ,label) in enumerate(data0_train):
        x , label = x.to(device), label.to(device)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            y1 = model(x)
            loss = criteon(y1,label) 
        _, preds1 = torch.max(F.softmax(y1,dim=1), 1)
        #AMP优化
        scaler.scale(loss).backward()
#         loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = 5, norm_type=2)  #梯度剪裁
        scaler.step(optimizer)
#         optimizer.step()
        scaler.update()
        train_loss1 += loss.item()*batch_size
        train_acc1 += torch.sum(preds1 == label.data).double()
        number = number + 1
    time_two = time.time()             #标记训练结束时间戳
    #输出训练一轮所需时间，用于分析对比
    print("The Model-Train-Time spent  %d min %.2f s"%((time_two-time_one)//60,(time_two-time_one)%60))
    #计算训练时候的平均损失和平均准确率
    train_loss = train_loss1 / (number*batch_size)
    train_acc = train_acc1 / (number*batch_size)
    #计算测试时候的平均损失和平均准确率
    val_acc, val_loss = evalute_(model, data1_val)
    
    train_acc = train_acc.cpu()
    val_acc = val_acc.cpu()
    print('Accuracy : Train is {} , Valid is {} ;  Loss : Train is  {} ,Valid is {}'.format(train_acc, val_acc, train_loss , val_loss))
    #如果你不需要训练以及验证的准确率和损失值，你可以注释这下面的两行，它们不是非必须的，理论上只存在于汇报和论文中
    dataframe = pd.DataFrame(columns = [epoch,train_acc,train_loss,val_acc, val_loss])
    dataframe.to_csv(data_csv_path,line_terminator="\n",mode='a',index=False,sep=',')
    if val_acc > best_acc:
        print("覆盖最好的模型...")
        best_acc = val_acc 
        checkpoint = {
            'epoch': epoch,
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'best_acc':best_acc
        }
        torch.save(checkpoint,model_save_path)
#     time_three = time.time() 
#     print("测试花费时间",time_three-time_two)
    scheduler.step()  #动态学习率更新 
#如果你不是非必须，我建议你尽量不要使用n折交叉验证，使用数据增强可能效果更优于它。

epoch: 0
The Model-Train-Time spent  0 min 4.24 s
Accuracy : Train is 0.18098958333333331 , Valid is 0.2265625 ;  Loss : Train is  2.010319709777832 ,Valid is 1.8587390184402466
覆盖最好的模型...
epoch: 1
The Model-Train-Time spent  0 min 3.09 s
Accuracy : Train is 0.26171875 , Valid is 0.2734375 ;  Loss : Train is  1.8057518005371094 ,Valid is 1.7887393236160278
覆盖最好的模型...
epoch: 2
The Model-Train-Time spent  0 min 3.26 s
Accuracy : Train is 0.3411458333333333 , Valid is 0.296875 ;  Loss : Train is  1.6998380025227864 ,Valid is 1.7980130910873413
覆盖最好的模型...
epoch: 3
The Model-Train-Time spent  0 min 3.12 s
Accuracy : Train is 0.43359375 , Valid is 0.3359375 ;  Loss : Train is  1.5902442932128906 ,Valid is 1.7264536619186401
覆盖最好的模型...
epoch: 4
The Model-Train-Time spent  0 min 3.16 s
Accuracy : Train is 0.50390625 , Valid is 0.359375 ;  Loss : Train is  1.4821408589680989 ,Valid is 1.6538596153259277
覆盖最好的模型...
epoch: 5
The Model-Train-Time spent  0 min 3.15 s
Accuracy : Train is 0.546875 , 

The Model-Train-Time spent  0 min 3.17 s
Accuracy : Train is 0.9010416666666666 , Valid is 0.640625 ;  Loss : Train is  0.4805424263079961 ,Valid is 1.0631704330444336
epoch: 47
The Model-Train-Time spent  0 min 3.22 s
Accuracy : Train is 0.8958333333333333 , Valid is 0.6640625 ;  Loss : Train is  0.48806901276111603 ,Valid is 1.068780779838562
epoch: 48
The Model-Train-Time spent  0 min 3.16 s
Accuracy : Train is 0.8958333333333333 , Valid is 0.671875 ;  Loss : Train is  0.4824308753013611 ,Valid is 0.9542477130889893
epoch: 49
The Model-Train-Time spent  0 min 3.32 s
Accuracy : Train is 0.9153645833333333 , Valid is 0.6328125 ;  Loss : Train is  0.46016345421473187 ,Valid is 1.126114845275879
epoch: 50
The Model-Train-Time spent  0 min 3.18 s
Accuracy : Train is 0.9088541666666666 , Valid is 0.6484375 ;  Loss : Train is  0.4726458191871643 ,Valid is 1.0803163051605225
epoch: 51
The Model-Train-Time spent  0 min 3.19 s
Accuracy : Train is 0.90625 , Valid is 0.6328125 ;  Loss : Train i

The Model-Train-Time spent  0 min 3.58 s
Accuracy : Train is 0.9375 , Valid is 0.6484375 ;  Loss : Train is  0.4261930088202159 ,Valid is 1.0523526668548584
epoch: 94
The Model-Train-Time spent  0 min 3.13 s
Accuracy : Train is 0.93359375 , Valid is 0.6875 ;  Loss : Train is  0.41095713277657825 ,Valid is 1.0068812370300293
epoch: 95
The Model-Train-Time spent  0 min 3.91 s
Accuracy : Train is 0.921875 , Valid is 0.6484375 ;  Loss : Train is  0.4231334924697876 ,Valid is 1.0679595470428467
epoch: 96
The Model-Train-Time spent  0 min 3.43 s
Accuracy : Train is 0.92578125 , Valid is 0.6640625 ;  Loss : Train is  0.416875163714091 ,Valid is 1.0133442878723145
epoch: 97
The Model-Train-Time spent  0 min 3.14 s
Accuracy : Train is 0.91796875 , Valid is 0.6796875 ;  Loss : Train is  0.4454179455836614 ,Valid is 1.0193049907684326
epoch: 98
The Model-Train-Time spent  0 min 3.19 s
Accuracy : Train is 0.9296875 , Valid is 0.6875 ;  Loss : Train is  0.4248237907886505 ,Valid is 1.01485669612884

The Model-Train-Time spent  0 min 3.15 s
Accuracy : Train is 0.9596354166666666 , Valid is 0.7109375 ;  Loss : Train is  0.2880546301603317 ,Valid is 0.9192308187484741
epoch: 141
The Model-Train-Time spent  0 min 3.22 s
Accuracy : Train is 0.9622395833333333 , Valid is 0.671875 ;  Loss : Train is  0.28013312816619873 ,Valid is 0.9822475910186768
epoch: 142
The Model-Train-Time spent  0 min 3.20 s
Accuracy : Train is 0.953125 , Valid is 0.703125 ;  Loss : Train is  0.2913198098540306 ,Valid is 0.9781723618507385
epoch: 143
The Model-Train-Time spent  0 min 3.23 s
Accuracy : Train is 0.9739583333333333 , Valid is 0.6484375 ;  Loss : Train is  0.2775547405083974 ,Valid is 1.1018794775009155
epoch: 144
The Model-Train-Time spent  0 min 3.13 s
Accuracy : Train is 0.95703125 , Valid is 0.6796875 ;  Loss : Train is  0.28682836641867954 ,Valid is 0.9698790311813354
epoch: 145
The Model-Train-Time spent  0 min 3.16 s
Accuracy : Train is 0.9739583333333333 , Valid is 0.6796875 ;  Loss : Train i

Accuracy : Train is 0.9895833333333333 , Valid is 0.6953125 ;  Loss : Train is  0.15016522258520126 ,Valid is 1.0228861570358276
epoch: 187
The Model-Train-Time spent  0 min 3.15 s
Accuracy : Train is 0.9908854166666666 , Valid is 0.671875 ;  Loss : Train is  0.16250460346539816 ,Valid is 0.9563975930213928
epoch: 188
The Model-Train-Time spent  0 min 3.20 s
Accuracy : Train is 0.9856770833333333 , Valid is 0.6953125 ;  Loss : Train is  0.1625132163365682 ,Valid is 0.9922624230384827
epoch: 189
The Model-Train-Time spent  0 min 3.27 s
Accuracy : Train is 0.9830729166666666 , Valid is 0.6953125 ;  Loss : Train is  0.16684389611085257 ,Valid is 0.898087739944458
epoch: 190
The Model-Train-Time spent  0 min 3.24 s
Accuracy : Train is 0.9895833333333333 , Valid is 0.703125 ;  Loss : Train is  0.15381529678901038 ,Valid is 0.8995522260665894
epoch: 191
The Model-Train-Time spent  0 min 3.12 s
Accuracy : Train is 0.9869791666666666 , Valid is 0.6953125 ;  Loss : Train is  0.16441532721122107

In [14]:
#模型在末尾处，另一模型提交，导致时间成本增高